# Reverse-time and Kirchhoff migration

By Adonis JAMAL and Jean-Vincent MARTINI

The goal is to generate synthetic data and plot the images obtained by Reverse-Time migration in diﬀerent two-dimensional conﬁgurations, and to study the resolution and stability properties of the method with respect to measurement noise, and to compare with other methods.

---
## 1) Preliminaries

We assume that the medium has speed of propagation $c_0 = 1$. The homogeneous two-dimensional Green's function $\hat{G}_0(\omega, x, y)$ is solution of $$\Delta_x \hat{G}_0 + \omega^2 \hat{G}_0 = -\delta(x-y), \quad x \in \mathbb{R}^2$$ with the Sommerfeld radiation condition. It is given by $$\hat{G}_0(\omega, x, y) = \frac{i}{4} H_0^{(1)}(\omega |x-y|)$$ where $H_0^{(1)}$ is the Hankel function $$H_0^{(1)}(s) = J_0(s) + i Y_0(s)$$ and $J_0$ is the Bessel function of the first kind of order zero and $Y_0$ is the Bessel function of the second kind of order zero.

In [60]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import hankel1, j0
from tqdm import tqdm
from ipywidgets import interact, FloatSlider

import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

---
## 2) Time-harmonic localization - full aperture

Consider $N$ transducers on a circular array centered at $0$ with radius $R_0$. Consider a point-like reflector at $x_{ref}$.

Generate the dataset, i.e. the matrix of the time-harmonic amplitudes $\hat{u}_{rs}(\omega)$ recorded by the $r$-th receiver when the $s$-th source emits a time-harmonic signal with unit amplitude and frequency $\omega$. Use a Born approximation for the reflector to generate the data.

Give the forms of the (two-dimensional) RT and KM imaging functional $\mathcal{I}_{RT}(x)$ and $\mathcal{I}_{KM}(x)$, and plot them using the data set.

Compare the focal spot with the theoretical function $J_0^2(\omega |x-x_{ref}|)$ (give a proof of this formula).

Use $\omega = 2\pi$, $R_0 = 100$, $N = 100$, and $x_{ref} = (10, 20)$, and play with the numbers (in particular, move the reflector).

<blockquote>

Since the transducers are placed on a circular array centered at $\mathbf{0}$ with radius $R_0$, the position of each transducer can be expressed in the form $\bm{x}_t = R_0 (\cos \theta_t, \sin \theta_t)$, where $\theta_t=\frac{2\pi (t-1)}{N}$ for $t=1,2,...,N$.

When a source $s$ emits a time-harmonic signal at frequency $\omega$, the signal recorded at the position $\bm{x}$ can be expressed using the Green's function as:
$$\hat{u}_{s}(\omega, \bm{x}) = \hat{G}_0(\omega, \bm{x}, \bm{x}_s) = \frac{i}{4} H_0^{(1)}(\omega |\bm{x}-\bm{x}_s|).$$

A point-like reflector at $\bm{x}_{ref}$ is modeled with $\delta(\bm{x}-\bm{x}_{ref})$. Using the Born approximation, the filed measured at the receiver $r$ due to the source $s$ can be expressed as:
$$\hat{u}_{rs}(\omega) = \omega^2 \int_{\mathbb{R}^2} \hat{G}_0(\omega, \bm{x}_r, \bm{x}) \delta(\bm{x}-\bm{x}_{\text{ref}}) \hat{G}_0(\omega, \bm{x}, \bm{x}_s) \, d\bm{x} = \omega^2 \hat{G}_0(\omega, \bm{x}_r, \bm{x}_{\text{ref}}) \hat{G}_0(\omega, \bm{x}_{\text{ref}}, \bm{x}_s).$$

Using the explicit expression of the Green's function, we have:
\begin{align*}
\hat{u}_{rs}(\omega)
&= \omega^2 \left(\frac{i}{4}\right)^2 H_0^{(1)}(\omega |\bm{x}_r - \bm{x}_{\text{ref}}|) H_0^{(1)}(\omega |\bm{x}_{\text{ref}} - \bm{x}_s|) \\
&= -\frac{\omega^2}{16} H_0^{(1)}(\omega |\bm{x}_r - \bm{x}_{\text{ref}}|) H_0^{(1)}(\omega |\bm{x}_{\text{ref}} - \bm{x}_s|)
\end{align*}

Since our transducers are both sources and receivers, we can write the $\hat{u}_{rs}(\omega)$ as a $N \times N$ data matrix where each entry corresponds to the measurement at receiver $r$ due to source $s$. By defining $v_t = H_0^{(1)}(\omega |\bm{x}_t - \bm{x}_{\text{ref}}|)$, for $t=1,2,...,N$, since our indexes $r$ and $s$ are interchangeable, we can express the data matrix as:
$$\hat{U}(\omega) = -\frac{\omega^2}{16} \mathbf{v} \mathbf{v}^T,$$
where $\mathbf{v} = (v_1, v_2, ..., v_N)^T$.

Reverse-Time (RT) migration and Kirchhoff Migration (KM) are two imaging techniques used to reconstruct the location of the reflector from the measured data. The Kirchhoff migration is obtained as a simplification of the reverse-time imaging function in which we take $\hat{G}_0 (\omega, \bm{x}, \bm{y}) \approx \exp{[i\omega \mathcal{T}(\bm{x}, \bm{y})]}$, $\mathcal{T}(\bm{x}, \bm{y})$ is the travel time from $\bm{x}$ to $\bm{y}$. The imaging functionals for RT and KM can be defined as follows:
$$\mathcal{I}_{RT}(\bm{x}) = \frac{1}{2\pi} \int_{-\infty}^{\infty} d\omega \sum_{r=1}^N \sum_{s=1}^N \overline{\hat{G}_0(\omega, \bm{x}, \bm{x}_r)} \hat{u}_{rs}(\omega) \overline{\hat{G}_0(\omega, \bm{x}_s, \bm{x})},$$
$$\mathcal{I}_{KM}(\bm{x}) = \frac{1}{2\pi} \int_{-\infty}^{\infty} d\omega \sum_{r=1}^N \sum_{s=1}^N e^{-i\omega|\bm{x} - \bm{x}_r|} \hat{u}_{rs}(\omega) e^{-i\omega|\bm{x}_s - \bm{x}|}.$$

At a single frequency $\omega$, we can simplify these functionals to:
$$\mathcal{I}_{RT}(\bm{x}) = \sum_{r=1}^N \sum_{s=1}^N \overline{\hat{G}_0(\omega, \bm{x}, \bm{x}_r)} \hat{u}_{rs}(\omega) \overline{\hat{G}_0(\omega, \bm{x}_s, \bm{x})},$$
$$\mathcal{I}_{KM}(\bm{x}) = \sum_{r=1}^N \sum_{s=1}^N e^{-i\omega|\bm{x} - \bm{x}_r|} \hat{u}_{rs}(\omega) e^{-i\omega|\bm{x}_s - \bm{x}|}.$$

By substituting the expression of $\hat{u}_{rs}(\omega)$ into the imaging functionals, we can express them as a square magnitude of a sum involving the function $H_0^{(1)}$ and its complex conjugate, up to a multiplicative constant. Specifically, we have:
$$\mathcal{I}_{RT}(\bm{x}) \propto \left| \sum_{t=1}^N \overline{H_0^{(1)}(\omega |\bm{x}-\bm{x}_t|)} H_0^{(1)}(\omega |\bm{x}_t - \bm{x}_{\text{ref}}|) \right|^2,$$
$$\mathcal{I}_{KM}(\bm{x}) \propto \left| \sum_{t=1}^N e^{-i\omega|\bm{x} - \bm{x}_t|} H_0^{(1)}(\omega |\bm{x}_t - \bm{x}_{\text{ref}}|) \right|^2.$$

Since the Hankel function $H_0^{(1)}$ can be approximated by $H_0^{(1)}(s) \approx \sqrt{\frac{2}{\pi s}} e^{i(s - \frac{\pi}{4})}$, the term $e^{-i\omega|\bm{x} - \bm{x}_t|}$ in the KM functional can be seen as an approximation of $\overline{H_0^{(1)}(\omega |\bm{x}-\bm{x}_t|)}$ in the RT functional. Therefore, in this particular configuration, we can see that $\mathcal{I}_{RT}(\bm{x}) \approx \mathcal{I}_{KM}(\bm{x})$ for all $\bm{x}$.

Next, we will prove that, at the focal spot, the imaging functional behaves like $\mathcal{I}(\bm{x}) \sim J_0^2(\omega |\bm{x}-\bm{x}_{\text{ref}}|)$, where $J_0$ is the Bessel function of the first kind of order zero. 

As the number of transducers $N\to\infty$, uniformly distributed on the circle $\Gamma_{R_0}$ of radius $R_0$, the discrete sum in the imaging functional converges to the following integral:
$$
\mathcal{I}(\bm{x})\propto
\left|
\int_{\Gamma_{R_0}}
\overline{H_0^{(1)}\big(\omega|\bm{x}-\bm{y}|\big)}
H_0^{(1)}\big(\omega|\bm{x}_{\mathrm{ref}}-\bm{y}|\big)
\,d\sigma(\bm{y})
\right|^2,
$$
where $d\sigma$ denotes the arc-length measure on $\Gamma_{R_0}$.

Using the far-field approximation $R_0=|\bm{y}|>>1$, the phase terms dominate: $$ \overline{H_0^{(1)}\big(\omega|\bm{x}-\bm{y}|\big)} H_0^{(1)}\big(\omega|\bm{x}_{\mathrm{ref}}-\bm{y}|\big) \approx \frac{2}{\pi \omega R_0} e^{-i\omega|\bm{x}-\bm{y}|} e^{i\omega|\bm{x}_{\mathrm{ref}}-\bm{y}|}$$

We approximate the phase difference: $$|\bm{x}_\mathrm{ref}-\bm{y}| - |\bm{x}-\bm{y}| \approx \frac{y}{|\bm{y}|} \cdot (\bm{x}-\bm{x}_{\mathrm{ref}})$$

Let $y = R_0 (\cos \theta, \sin \theta)$ and $x - x_{\mathrm{ref}} = \Delta x$. Then the phase is $k \cos (\theta - \phi) |\Delta x|$ where $\phi$ is the angle of $\Delta x$. Therefore, the integral over the circle becomes: $$ \int_0^{2\pi} e^{i \omega \frac{y}{R_0} \cdot (x - x_\mathrm{ref})} d\theta = 2\pi J_0(\omega |\bm{x}-\bm{x}_{\mathrm{ref}}|)$$

Squaring the result gives us the final resolution profile: $$\mathcal{I}(\bm{x}) \propto J_0^2(\omega |\bm{x}-\bm{x}_{\mathrm{ref}}|)$$

<blockquote>

In [ ]:
# Parameters
N = 100             # Number of transducers
w = 2 * np.pi       # Frequency
R0 = 100            # Array radius
x_ref = (10, 20)    # Point-like reflector position

theta_t = np.linspace(0, 2 * np.pi, N, endpoint=False)                  # Angles for transducers
x_t = np.stack((R0 * np.cos(theta_t), R0 * np.sin(theta_t)), axis=-1)   # Transducer positions

# Grid parameters for imaging
n = 200
xlim = (-125, 125)
zlim = (-125, 125)
x = np.linspace(xlim[0], xlim[1], n)
z = np.linspace(zlim[0], zlim[1], n)
X, Z = np.meshgrid(x, z)

In [ ]:
# Green's function
def GreenRT(dist, omega=w):
    """Green's function for Reverse-Time Migration."""
    return (1j / 4) * hankel1(0, omega * dist)

def GreenKM(dist, omega):
    """Green's function for Kirchhoff Migration."""
    return np.exp(-1j * omega * dist)

In [ ]:
# Generate data matrix using Born approximation
dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
v_t = GreenRT(dist_t_ref, w)
U = -(w**2 / 16) * np.outer(v_t, v_t)

# Compute imaging functionals
I_RT = np.zeros_like(X, dtype=float)
I_KM = np.zeros_like(X, dtype=float)

for i in range(n):
    for j in range(n):
        X_ij = np.array([X[i, j], Z[i, j]])

        # Distances from transducers to current imaging point
        dists = np.linalg.norm(x_t - X_ij, axis=1)

        # Reverse-Time Migration
        g_rt = GreenRT(dists, w)
        vec_rt = np.conj(g_rt)
        sum_RT = vec_rt @ U @ vec_rt.T
        I_RT[i, j] = np.abs(sum_RT) ** 2

        # Kirchhoff Migration
        vec_km = GreenKM(dists, w)
        sum_KM = vec_km @ U @ vec_km.T
        I_KM[i, j] = np.abs(sum_KM) ** 2

dist = np.sqrt((X - x_ref[0]) ** 2 + (Z - x_ref[1]) ** 2)
I_TH = j0(w * dist) ** 2

# Normalize for better visualization
I_RT /= I_RT.max()
I_KM /= I_KM.max()
I_TH /= I_TH.max()

In [ ]:
# Plotting the results
fig, axes = plt.subplots(1, 3, figsize=(12, 5))

im0 = axes[0].imshow(
    I_RT, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot"
)

axes[0].set_title("Reverse Time Migration")
axes[0].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
axes[0].scatter(x_t[:, 0], x_t[:, 1], color="white", marker="x", label="Transducers")
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(
    I_KM, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot"
)

axes[1].set_title("Kirchhoff Migration")
axes[1].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
axes[1].scatter(x_t[:, 0], x_t[:, 1], color="white", marker="x", label="Transducers")
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(
    I_TH, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot"
)

axes[2].set_title("Theoretical Imaging Functional")
axes[2].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
axes[2].scatter(x_t[:, 0], x_t[:, 1], color="white", marker="x", label="Transducers")
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
ref_positions = [(10, 20), (0, 0), (50, 50), (99, 0), (0, 100), (40, 80)]
num_rows = len(ref_positions) // 2
num_cols = 4  # 2 plots per position * 2 positions per row

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 12))

for idx, x_ref in enumerate(ref_positions):
    # Generate data matrix using Born approximation
    dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
    v_t = GreenRT(dist_t_ref, w)
    U = -(w**2 / 16) * np.outer(v_t, v_t)

    # Imaging
    I_RT = np.zeros_like(X, dtype=float)

    for i in range(n):
        for j in range(n):
            X_ij = np.array([X[i, j], Z[i, j]])

            # Distances from transducers to current imaging point
            dists = np.linalg.norm(x_t - X_ij, axis=1)

            # Reverse-Time migration
            g_rt = GreenRT(dists, w)
            vec_rt = np.conj(g_rt)
            sum_RT = vec_rt @ U @ vec_rt.T
            I_RT[i, j] = np.abs(sum_RT) ** 2

    # Theoretical resolution
    dist_grid_ref = np.sqrt((X - x_ref[0]) ** 2 + (Z - x_ref[1]) ** 2)
    I_TH = j0(w * dist_grid_ref) ** 2

    # Normalize for better visualization
    I_RT /= I_RT.max()
    I_TH /= I_TH.max()

    # Calculate grid location
    row_idx = idx // 2
    col_start = (idx % 2) * 2  # Starts at 0 for even idx, 2 for odd idx
    ax_rt = axes[row_idx, col_start]
    ax_th = axes[row_idx, col_start + 1]

    ax_rt.set_title(f"Position {x_ref}: Reverse Time Migration")
    ax_rt.imshow(I_RT, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
    ax_rt.scatter(x_t[:, 0], x_t[:, 1], color="white", marker="x", label="Transducers")
    ax_rt.scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")

    ax_th.set_title(f"Position {x_ref}: Theoretical Imaging Functional")
    ax_th.imshow(I_TH, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
    ax_th.scatter(x_t[:, 0], x_t[:, 1], color="white", marker="x", label="Transducers")
    ax_th.scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")

plt.tight_layout()
plt.show()

In [ ]:
# FAR FIELD APPROXIMATION

# Parameters
N = 2000        # Number of transducers
w = 2 * np.pi   # Frequency
R0 = 1000       # Array radius
x_ref = (1, 2)  # Point-like reflector position

theta_t = np.linspace(0, 2 * np.pi, N, endpoint=False)                  # Angles for transducers
x_t = np.stack((R0 * np.cos(theta_t), R0 * np.sin(theta_t)), axis=-1)   # Transducer positions

# Grid parameters for imaging
n = 200
xlim = (-50, 50)
zlim = (-50, 50)
x = np.linspace(xlim[0], xlim[1], n)
z = np.linspace(zlim[0], zlim[1], n)
X, Z = np.meshgrid(x, z)

# Generate data matrix using Born approximation
dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
v_t = GreenRT(dist_t_ref, w)
U = -(w**2 / 16) * np.outer(v_t, v_t)

# Compute imaging functionals
I_RT = np.zeros_like(X, dtype=float)
I_KM = np.zeros_like(X, dtype=float)

for i in range(n):
    for j in range(n):
        X_ij = np.array([X[i, j], Z[i, j]])

        dists = np.linalg.norm(x_t - X_ij, axis=1)

        # Reverse-Time migration
        g_rt = GreenRT(dists, w)
        vec_rt = np.conj(g_rt)
        sum_RT = vec_rt @ U @ vec_rt.T
        I_RT[i, j] = np.abs(sum_RT) ** 2

        # Kirchhoff Migration
        vec_KM = GreenKM(dists, w)
        sum_KM = vec_KM @ U @ vec_KM.T
        I_KM[i, j] = np.abs(sum_KM) ** 2

# Theoretical resolution
dist_grid_ref = np.sqrt((X - x_ref[0]) ** 2 + (Z - x_ref[1]) ** 2)
I_TH = j0(w * dist_grid_ref) ** 2

# Normalize for better visualization
I_RT /= I_RT.max()
I_KM /= I_KM.max()
I_TH /= I_TH.max()

# Plot results
fig, axes = plt.subplots(1, 3, figsize=(12, 5))

im0 = axes[0].imshow(I_RT, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[0].set_title("Reverse Time Migration")
axes[0].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
plt.colorbar(im0, ax=axes[0])

im1 = axes[1].imshow(I_KM, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[1].set_title("Kirchhoff Migration")
axes[1].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
plt.colorbar(im1, ax=axes[1])

im2 = axes[2].imshow(I_TH, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[2].set_title("Theoretical Imaging Functional")
axes[2].scatter(x_ref[0], x_ref[1], color="blue", marker="*", label="Reflector")
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

<blockquote>

TODO: Interpret the result & add interactive plots to test for different parameters.

</blockquote>

---
## 3) Time-harmonic localization - partial aperture

We use the convention $x = (x, z)$.

Consider $N$ receivers $(x_r)_{r=1,\dots,N}$ on a regular linear array (along the $x$-direction) centered at $0$ with length $R_0$ (i.e. $x_r = (x_r, 0), x_r = - R_0/2 + R_0 (r - 1) / (N-1)$).

Generate the data set, i.e. the matrix of the time-harmonic amplitudes $\hat{u}_{rs}(\omega)$ recorded by the $r$-th receiver when the $s$-th source emits a time-harmonic signal with unit amplitude and frequency $\omega$. Use a Born approximation for the point-like reflector.

Plot the (two-dimensional) RT and KM imaging functional $\mathcal{I}_{RT}(x)$ and $\mathcal{I}_{KM}(x)$ using the data set. Plot also the MUSIC-type imaging functional $$\mathcal{I}_{MU}(x) = |\langle \hat{g}(\omega, x), v_1 \rangle|^2,$$ where $\hat{g}(\omega, x)$ is the vector of the Green's functions from the array to the search point $x$: $$\hat{g}(\omega, x) = (\hat{G}_0(\omega, x, x_r))_{r=1,\dots,N}$$ and $v_1$ is the first singular vector of the response matrix $\hat{u}$ (MUSIC means MUltiple SIgnal Classification).

Use $\omega = 2\pi$, $R_0 = 100$, $N = 100$, and $x_{ref} = (10, 20)$, and play with the numbers (Reduce $R_0$, move the reflector).

Look at the focal spot in the (cross-range) $x$-direction and compare with the theoretical function $\text{sinc}^2(\pi |x - x_{ref})|/r_c)$, with $r_c = \lambda |x_{ref}| / R_0$ and $\lambda = 2\pi / \omega$.

Look at the focal spot in the (range) $z$-direction and compare with the theoretical function $|\int_{0}^{1} \exp(-i \frac{\pi}{2} s^2 \frac{|z-z_{ref})}{r_l}) ds|^2$, with $r_l = 2 \lambda |x_{ref}|^2 / R_0^2$.

In [ ]:
# Parameters
N = 100             # Number of transducers
w = 2 * np.pi       # Frequency
R0 = 100            # Array radius
x_ref = (10, 20)    # Point-like reflector position

x_coords = np.linspace(-R0 / 2, R0 / 2, N)          # Transducer x-coordinates
x_t = np.stack((x_coords, np.zeros(N)), axis=-1)    # Transducer positions

# Grid parameters for imaging
n = 200
xlim = (-125, 125)
zlim = (-125, 125)
x = np.linspace(xlim[0], xlim[1], n)
z = np.linspace(zlim[0], zlim[1], n)
X, Z = np.meshgrid(x, z)

In [ ]:
# Generate data matrix using Born approximation
dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
v_t = GreenRT(dist_t_ref, w)
U = -(w**2 / 16) * np.outer(v_t, v_t)

# Compute imaging functionals
I_RT = np.zeros_like(X, dtype=float)
I_KM = np.zeros_like(X, dtype=float)
I_MU = np.zeros_like(X, dtype=float)

# First singular vector for MUSIC-type functional
u_svd, s_svd, vh_svd = np.linalg.svd(U)
v1 = vh_svd.conj()[0, :]

for i in range(n):
    for j in range(n):
        X_ij = np.array([X[i, j], Z[i, j]])
        dists = np.linalg.norm(x_t - X_ij, axis=1)

        # Reverse-Time migration
        g_rt = GreenRT(dists, w)
        vec_rt = np.conj(g_rt)
        sum_RT = vec_rt @ U @ vec_rt.T
        I_RT[i, j] = np.abs(sum_RT) ** 2

        # Kirchhoff Migration
        vec_KM = GreenKM(dists, w)
        sum_KM = vec_KM @ U @ vec_KM.T
        I_KM[i, j] = np.abs(sum_KM) ** 2

        # MUSIC
        g_music = GreenRT(dists, w)
        proj = np.vdot(g_music, v1)
        I_MU[i, j] = np.abs(proj) ** 2

# Normalize Images
I_RT /= I_RT.max()
I_KM /= I_KM.max()
I_MU /= I_MU.max()

# Theoretical resolution
lam = 2 * np.pi / w
norm_x_ref = np.linalg.norm(x_ref)
r_c = (lam * norm_x_ref) / R0
r_l = (2 * lam * norm_x_ref**2) / (R0**2)

# Cross-Range Slice
z_idx = np.argmin(np.abs(z - x_ref[1]))
x_slice_vals = X[z_idx, :]
rt_slice_x = I_RT[z_idx, :]
th_slice_x = np.sinc((x_slice_vals - x_ref[0]) / r_c) ** 2

# Range Slice
x_idx = np.argmin(np.abs(x - x_ref[0]))
z_slice_vals = Z[:, x_idx]
rt_slice_z = I_RT[:, x_idx]

# Theoretical Integral for Range
th_slice_z = []
s_dummy = np.linspace(0, 1, 1000)

for z_val in z_slice_vals:
    phase_factor = (np.pi / 2) * (np.abs(z_val - x_ref[1]) / r_l)
    integrand = np.exp(-1j * phase_factor * (s_dummy**2))
    val = np.abs(np.trapezoid(integrand, s_dummy)) ** 2
    th_slice_z.append(val)

th_slice_z = np.array(th_slice_z)
th_slice_z /= th_slice_z.max()

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

axes[0, 0].imshow(I_RT, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[0, 0].set_title("Reverse Time (RT)")
axes[0, 0].scatter(*x_ref, c="blue", marker="*")
axes[0, 0].scatter(x_t[:, 0], x_t[:, 1], c="white", s=1)

axes[0, 1].imshow(I_KM, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[0, 1].set_title("Kirchhoff (KM)")
axes[0, 1].scatter(*x_ref, c="blue", marker="*")
axes[0, 1].scatter(x_t[:, 0], x_t[:, 1], c="white", s=1)

axes[0, 2].imshow(I_MU, extent=(xlim[0], xlim[1], zlim[0], zlim[1]), origin="lower", cmap="hot")
axes[0, 2].set_title("MUSIC")
axes[0, 2].scatter(*x_ref, c="blue", marker="*")
axes[0, 2].scatter(x_t[:, 0], x_t[:, 1], c="white", s=1)

# Cross-Range
axes[1, 0].plot(x_slice_vals, rt_slice_x, label="RT (Simulated)")
axes[1, 0].plot(x_slice_vals, th_slice_x, "--", label="Theory (Sinc^2)")
axes[1, 0].set_title("Cross-Range Resolution (x-axis)")
axes[1, 0].set_xlabel("x")
axes[1, 0].legend()
axes[1, 0].grid(True)

# Range
axes[1, 1].plot(z_slice_vals, rt_slice_z, label="RT (Simulated)")
axes[1, 1].plot(z_slice_vals, th_slice_z, "--", label="Theory (Fresnel Integral)")
axes[1, 1].set_title("Range Resolution (z-axis)")
axes[1, 1].set_xlabel("z")
axes[1, 1].legend()
axes[1, 1].grid(True)

axes[1, 2].axis("off")

plt.tight_layout()
plt.show()

<blockquote>

TODO: Interpret the result & test for different parameters.

</blockquote>

---
## 4) Time-dependent localization - partial aperture

Consider $N$ receivers on a linear array (along the $x$-direction) centered at $0$ with length $R_0$.

We now assume that the sources emit a broadband signal with $\hat{f}(\omega) = 1_{[\omega_0 - B, \omega_0 + B]}(\omega)$.

Generate the data set, i.e. the matrices of the time-harmonic amplitudes $\hat{d}_{rs}(\omega)$ recorded by the $r$-th receiver when the $s$-th source emits a time-harmonic signal with unit amplitude and frequency $\omega$, for $\omega$ sampled in $[\omega_0 - B, \omega_0 + B]$.

Plot the (two-dimensional) RT and KM imaging functional $\mathcal{I}_{RT}(x)$ and $\mathcal{I}_{KM}(x)$ using the data set.

Use $\omega_0 = 2\pi$, $B = 0.05\omega_0$, $R_0 = 20$, $N = 40$, and $x_{ref} = (0, 100)$, and play with the numbers (Reduce the bandwidth $B$ for instance).

Look at the focal spot in the (cross-range) $x$-direction and compare with the theoretical function $\text{sinc}^2(\pi |x - x_{ref})|/r_c)$, with $r_c = \lambda_0 |x_{ref}| / R_0$ and $\lambda_0 = 2\pi / \omega_0$.

Look at the focal spot in the (range) $z$-direction and compare with the theoretical function $|\operatorname{sinc}(2B |z-z_{ref}|)|$.

In [43]:
# Parameters
N = 40                                  # Number of receivers
w0 = 2 * np.pi                          # Center frequency
B = 0.05 * w0                           # Bandwidth
Nw = 41                                 # Number of frequencies samples in the band
ws = np.linspace(w0 - B, w0 + B, Nw)    # Frequency range

R0 = 20                                 # Array length
x_ref = (0, 100)                        # Point-like reflector position

# Transducer positions
x_coords = np.linspace(-R0 / 2, R0 / 2, N)
x_t = np.stack((x_coords, np.zeros(N)), axis=-1)

# Grid parameters for imaging
n = 200
xlim = (-50, 50)
zlim = (-150, 150)
x = np.linspace(xlim[0], xlim[1], n)
z = np.linspace(zlim[0], zlim[1], n)
X, Z = np.meshgrid(x, z)

# Theoretical Parameters
lambda0 = 2 * np.pi / w0
r_c = (lambda0 * x_ref[1]) / R0         # Cross-range resolution scale

In [44]:
Image_RT = np.zeros_like(X, dtype=complex)
Image_KM = np.zeros_like(X, dtype=complex)

# Distances from transducers to imaging grid
grid_points = np.stack((X, Z), axis=-1)
dists_grid = np.linalg.norm(grid_points[:, :, None, :] - x_t[None, None, :, :], axis=3)

for omega in ws:
    # Data Matrix (Born Approximation)
    dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
    v_t = GreenRT(dist_t_ref, omega)
    U = -(omega**2 / 16) * np.outer(v_t, v_t)
    
    # Reverse-Time Migration
    g_rt = GreenRT(dists_grid, omega)
    vec_rt = np.conj(g_rt)
    temp_rt = vec_rt @ U
    step_rt = np.sum(vec_rt * temp_rt, axis=2)
    Image_RT += step_rt
    
    # Kirchhoff Migration
    g_km = GreenKM(dists_grid, omega)
    vec_km = np.conj(g_km)
    temp_km = vec_km @ U
    step_km = np.sum(vec_km * temp_km, axis=2)
    Image_KM += step_km

# Intensity
I_RT = np.abs(Image_RT)**2
I_KM = np.abs(Image_KM)**2

# Normalize
I_RT /= I_RT.max()
I_KM /= I_KM.max()

In [45]:
# Theoretical cross-range resolution
z_idx = np.argmin(np.abs(z - x_ref[1]))
rt_slice_x = I_RT[z_idx, :]
th_slice_x = np.sinc((x - x_ref[0]) / r_c) ** 2

# Theoretical range resolution
x_idx = np.argmin(np.abs(x - x_ref[0]))
rt_slice_z = I_RT[:, x_idx]
dist_z = np.abs(z - x_ref[1])
th_slice_z = np.abs(np.sinc((2 * B * dist_z) / np.pi))**2

# Plot Results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Reverse-Time Migration Image
im1 = axes[0, 0].imshow(I_RT, extent=(*xlim, *zlim), origin="lower", cmap="hot")
axes[0, 0].set_title("Reverse Time Migration")
axes[0, 0].scatter(*x_ref, c="blue", marker="*", label="Reflector")
axes[0, 0].scatter(x_t[:, 0], x_t[:, 1], c="white", s=1, label="Transducers")
axes[0, 0].set_aspect('equal')
plt.colorbar(im1, ax=axes[0, 0])

# Kirchhoff Migration Image
im2 = axes[0, 1].imshow(I_KM, extent=(*xlim, *zlim), origin="lower", cmap="hot")
axes[0, 1].set_title("Kirchhoff Migration")
axes[0, 1].scatter(*x_ref, c="blue", marker="*", label="Reflector")
axes[0, 1].scatter(x_t[:, 0], x_t[:, 1], c="white", s=1, label="Transducers")
axes[0, 1].set_aspect('equal')
plt.colorbar(im2, ax=axes[0, 1])

# Cross-Range Slice
axes[1, 0].plot(x, rt_slice_x, label="RT Simulation")
axes[1, 0].plot(x, th_slice_x, "--", label="Theoretical Cross-Range")
axes[1, 0].set_title(f"Cross-Range Resolution (z={x_ref[1]})")
axes[1, 0].set_xlabel("x")
axes[1, 0].legend()
axes[1, 0].grid(True)

# Range Slice
axes[1, 1].plot(z, rt_slice_z, label="RT Simulation")
axes[1, 1].plot(z, th_slice_z, "--", label="Theoretical Range")
axes[1, 1].set_title(f"Range Resolution (x={x_ref[0]})")
axes[1, 1].set_xlabel("z")
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

<blockquote>

TODO: Interpret the result & test for different parameters.

</blockquote>

---
## 5) Stability with respect to measurement noise

Consider measurement noise. The recorded signals are $\hat{u}_{rs}(\omega) + W_{rs}^{(1)}(\omega) + i W_{rs}^{(2)}(\omega)$, where the noise terms $W_{rs}^{(1)}(\omega), W_{rs}^{(2)}(\omega)$ are independent and identically distributed Gaussian random variables with mean zero and variance $\sigma^2 /2$.

Revisit the previous questions in the presence of measurement noise and study the stability issue (up to you: establish the statistics of the localization error, the dependence of the standard deviation in the localization error with respect to the number of sensors, with respect to the bandwidth, and so on, compare MUSIC and RT, ...).

In [46]:
# Parameters
N = 100             # Number of transducers
w = 2 * np.pi       # Frequency
R0 = 100            # Array radius
x_ref = (10, 20)    # Point-like reflector position

x_coords = np.linspace(-R0 / 2, R0 / 2, N)          # Transducer x-coordinates
x_t = np.stack((x_coords, np.zeros(N)), axis=-1)    # Transducer positions

# Grid parameters for imaging
n = 100
xlim = (-125, 125)
zlim = (-125, 125)
x = np.linspace(xlim[0], xlim[1], n)
z = np.linspace(zlim[0], zlim[1], n)
X, Z = np.meshgrid(x, z)

In [47]:
# Grid Distances
grid_points = np.stack((X.flatten(), Z.flatten()), axis=1)
dists_grid = np.linalg.norm(grid_points[:, None, :] - x_t[None, :, :], axis=2) # (Pixels, N)

# Clean Data Matrix
dist_t_ref = np.linalg.norm(x_t - x_ref, axis=1)
v_t = (1j / 4) * hankel1(0, w * dist_t_ref)
U_clean = -(w**2 / 16) * np.outer(v_t, v_t)

In [ ]:
def add_noise(U, sigma):
    """Adds complex Gaussian noise with variance sigma^2."""
    noise_real = np.random.normal(0, sigma / np.sqrt(2), U.shape)
    noise_imag = np.random.normal(0, sigma / np.sqrt(2), U.shape)
    return U + (noise_real + 1j * noise_imag)


def compute_images(sigma):
    # Add Noise
    U_noisy = add_noise(U_clean, sigma)
    
    G_grid = (1j / 4) * hankel1(0, w * dists_grid)

    # MUSIC
    u_svd, s_svd, vh_svd = np.linalg.svd(U_noisy)
    v1 = vh_svd.conj()[0, :]
    I_MU_flat = np.abs(G_grid @ v1.T)**2

    # Reverse-Time Migration
    G_conj = np.conj(G_grid)
    temp_RT = G_conj @ U_noisy # (Pixels, N)
    I_RT_flat = np.abs(np.sum(temp_RT * G_conj, axis=1))**2
    
    return I_RT_flat.reshape(X.shape), I_MU_flat.reshape(X.shape)


def update(sigma):
    """Interactive update function to compute and plot images for a given noise level."""
    I_RT, I_MU = compute_images(sigma)
    
    # Normalize
    if I_RT.max() > 0: I_RT /= I_RT.max()
    if I_MU.max() > 0: I_MU /= I_MU.max()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # RT Plot
    im0 = axes[0].imshow(I_RT, extent=(*xlim, *zlim), origin="lower", cmap="hot")
    axes[0].set_title(f"RT Migration (Noise $\\sigma$={sigma:.2f})")
    axes[0].scatter(*x_ref, c="blue", marker="*")
    plt.colorbar(im0, ax=axes[0])
    
    # MUSIC Plot
    im1 = axes[1].imshow(I_MU, extent=(*xlim, *zlim), origin="lower", cmap="hot")
    axes[1].set_title(f"MUSIC (Noise $\\sigma$={sigma:.2f})")
    axes[1].scatter(*x_ref, c="blue", marker="*")
    plt.colorbar(im1, ax=axes[1])
    
    plt.tight_layout()
    plt.show()


# Base sigma on maximum signal amplitude
max_amp = np.max(np.abs(U_clean))
print(f"Max Amplitude of Signal: {max_amp:.5f}")

interact(update, sigma=FloatSlider(min=0.0, max=max_amp*10, step=max_amp/20, value=0.0, readout_format='.5f', description="Noise Level"));

Max Amplitude of Signal: 0.00078


interactive(children=(FloatSlider(value=0.0, description='Noise Level', max=0.007810844465629173, readout_form…

In [61]:
# Monte Carlo Simulation for Stability Analysis

# Search Grid focused around the reflector
search_radius = 20
n_search = 30
xs = np.linspace(x_ref[0] - search_radius, x_ref[0] + search_radius, n_search)
zs = np.linspace(x_ref[1] - search_radius, x_ref[1] + search_radius, n_search)
XS, ZS = np.meshgrid(xs, zs)
grid_points_search = np.stack((XS.flatten(), ZS.flatten()), axis=1)

# Green's Function Matrix
dists_search = np.linalg.norm(grid_points_search[:, None, :] - x_t[None, :, :], axis=2)
G_search = (1j / 4) * hankel1(0, w * dists_search)
G_search_conj = np.conj(G_search)

# Simulation
noise_levels = np.linspace(0, max_amp * 2, 20)
n_trials = 50 

rmse_rt = []
rmse_mu = []

for sigma in tqdm(noise_levels):
    errors_rt = []
    errors_mu = []
    
    for _ in range(n_trials):
        # Generate Noise
        noise_real = np.random.normal(0, sigma / np.sqrt(2), U_clean.shape)
        noise_imag = np.random.normal(0, sigma / np.sqrt(2), U_clean.shape)
        U_noisy = U_clean + (noise_real + 1j * noise_imag)
        
        # MUSIC Estimator
        u_svd, s_svd, vh_svd = np.linalg.svd(U_noisy)
        v1 = vh_svd.conj()[0, :]
        proj_mu = np.abs(G_search @ v1)**2
        est_mu = grid_points_search[np.argmax(proj_mu)]
        errors_mu.append(np.linalg.norm(est_mu - np.array(x_ref)))
        
        # Reverse-Time Migration Estimator
        temp_rt = G_search_conj @ U_noisy
        I_RT_search = np.abs(np.sum(temp_rt * G_search_conj, axis=1))**2
        est_rt = grid_points_search[np.argmax(I_RT_search)]
        errors_rt.append(np.linalg.norm(est_rt - np.array(x_ref)))
    
    # Calculate RMSE
    rmse_rt.append(np.sqrt(np.mean(np.array(errors_rt)**2)))
    rmse_mu.append(np.sqrt(np.mean(np.array(errors_mu)**2)))

100%|██████████| 20/20 [00:25<00:00,  1.29s/it]


In [62]:
plt.figure(figsize=(10, 6))
plt.plot(noise_levels, rmse_rt, '-o', label='Reverse Time (RT)')
plt.plot(noise_levels, rmse_mu, '-s', label='MUSIC')
plt.xlabel(r'Noise Standard Deviation ($\sigma$)')
plt.ylabel('RMSE of Location Estimate')
plt.title(f'Stability: RT vs MUSIC (Reflector at {x_ref})')
plt.legend()
plt.grid(True)
plt.show()

---
## References

[1] H. Ammari, J. Garnier, W. Jing, H. Kang, M. Lim, K. Sølna, and H. Wang, Mathematical and Statistical Methods for Multistatic Imaging, Lecture Notes in Mathematics, Vol. 2098, Springer, Berlin, 2013.